In [75]:
import sys, os
from os.path import join, abspath, exists, pardir
import tomlkit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table, join as tab_join, vstack
import copy
from numpy import array as a
from sklearn.neighbors import KernelDensity
import pycatch22
from tempfile import TemporaryDirectory
import subprocess
from astropy.stats import sigma_clipped_stats, sigma_clip
from supersmoother import SuperSmoother
import scipy.ndimage

import astropy.units as u
from astropy.timeseries import TimeSeries, aggregate_downsample
from astropy.time import TimeDelta, Time

from tqdm import tqdm

from lightcurve import ASASSN_Lightcurve
from utils import read_config, n_hist_bins, hist
from data_preparation import prepare_lc, plot_prepared
from metrics import fill_lc_gaps, calc_metrics, short_metric_names

In [76]:
import warnings
warnings.filterwarnings('ignore', message='.*dubious year')

In [77]:
cfg = read_config("config.toml")
data_dir = cfg["data_dir"]

In [78]:
cleaned_dir = join(abspath(join(data_dir,os.pardir)),"cleaned")

train_path = "/media/disk/asasn/initial_tabular/train.csv"

outdir = "out"
os.makedirs(outdir,exist_ok=True)

def out(fname): return join(outdir,fname)
def savefig(fname): plt.savefig(out(fname),dpi=300,bbox_inches="tight")
def load_old_lc(fname): return ASASSN_Lightcurve.from_dat_file(join(data_dir,fname))
def load_cleaned_lc(fname): return ASASSN_Lightcurve.from_cleaned_file(join(cleaned_dir,fname))
rng = np.random.default_rng(42)
training_data = Table.read(train_path)

# Train-Validation Split

In [79]:
from tensorflow.keras.utils import to_categorical

i = np.arange(len(training_data))
rng.shuffle(i)

classes = np.unique(training_data["class"])
class_d = dict(zip(classes,np.arange(len(classes))))

def class_to_num(c): return class_d[c]

f = np.vectorize(class_to_num)

training_data["class_id"] = f(training_data["class"])

# for c in [c for c in training_data.colnames if c not in ["ID","class","class_id"]]:
#     training_data[c] = (training_data[c] - np.mean(training_data[c])) / np.std(training_data[c])
v = np.stack([a(training_data[col].astype(float)) for col in ([c for c in training_data.colnames if c not in ["ID","class"]])]).T

nb_classes = len(classes)


x_train = v[i[:128]]
x_valid = v[i[128:256]]

for c in range(x_train.shape[1]):
    mean, std = np.mean(x_train[:,c]), np.std(x_train[:,c])
    x_train[:,c] = (x_train[:,c]-mean) / std
    x_valid[:,c] = (x_valid[:,c]-mean) / std

y_train = to_categorical(a(training_data[i[:128]]["class_id"]), nb_classes)
y_valid = to_categorical(a(training_data[i[128:256]]["class_id"]), nb_classes)

x_train, y_train

(array([[ 5.06011093e-01, -5.57584022e-01, -2.51315239e-01, ...,
         -4.58355104e-01, -5.87919217e-01,  5.75622468e-01],
        [ 5.20573047e-01, -4.64895592e-01,  1.23066953e+00, ...,
         -2.59192410e-04,  1.23372133e+00, -2.16632111e-01],
        [ 3.28773374e-01, -7.41681099e-01,  5.63813148e-01, ...,
         -1.22260723e-01, -3.69030093e-01,  1.76400434e+00],
        ...,
        [-7.10923834e-01,  6.95367309e-02,  4.55648606e-01, ...,
          4.29806643e+00, -9.28259626e-02,  1.79495178e-01],
        [-4.32416515e-01,  8.19331974e-01, -3.99957845e-01, ...,
         -4.76423721e-01,  3.04149625e-01,  5.75622468e-01],
        [ 4.39966485e-01, -7.42127264e-01,  3.75472508e-01, ...,
         -2.56851087e-01, -2.58258958e+00, -2.16632111e-01]],
       shape=(128, 25)),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.

# Model-building

In [80]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model


- Initial Layers: Build an MLP in Keras to process the input features.
- Custom Residual Block:
    - Using the Keras Functional API, create a block with at least two Dense layers with ReLU activations.
    - Implement a residual connection by adding the block’s input to its output (apply a linear projection with an extra Dense layer if the dimensions differ).
- Additional Skip Connection:
    - Implement an extra skip connection that bypasses one or more intermediate layers outside the residual block.
- Final Layers:
    - Add further Dense layers.
    - Include an output layer appropriate for the task (e.g., a single unit with sigmoid activation for binary classification).


In [81]:
x_train.shape

(128, 25)

In [82]:
inputs = Input(shape=(x_train.shape[1],))

# input MLP
di = Dense(x_train.shape[1],activation='relu')(inputs)
di = Dense(x_train.shape[1],activation='relu')(di)

# residual block
d = Dense(x_train.shape[1],activation='relu')(di)
d = Dense(x_train.shape[1],activation='relu')(d)
res = Add()([d,di])

# after residual
d2 = Dense(x_train.shape[1],activation='relu')(res)
d2 = Dense(x_train.shape[1],activation='relu')(d2)

# skip connection
comb = Add()([di,d2])

output = Dense(len(classes),activation="softmax")(comb)

model = Model(inputs=inputs,outputs=output)
model

<Functional name=functional_7, built=True>

In [83]:
model.save("sage_assignment_4_model.h5")

# Model Training

In [84]:
model.compile(loss='categorical_crossentropy', optimizer="ADAM", metrics=['accuracy'])

In [85]:
model.fit(x_train, y_train,epochs=250, verbose=1,)

Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.1365 - loss: 2.3370
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1646 - loss: 2.1527
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1917 - loss: 2.1436
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2708 - loss: 1.9853
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3094 - loss: 1.9323
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2896 - loss: 1.8937
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3521 - loss: 1.7819 
Epoch 8/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3531 - loss: 1.7612 
Epoch 9/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3510 - loss: 1.6994
Epoch 10/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4031 - loss: 1.6431 
Epoch 11/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3740 - loss: 1.6542 
Epoch 12/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3990 - los

In [86]:
t_loss, _ = model.evaluate(x_train,y_train)
v_loss, _ = model.evaluate(x_valid,y_valid)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 8.5754e-04
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6646 - loss: 2.7103


In [87]:
print(f"Number of parameters: {model.count_params()}")
print(f"Final training loss: {round(t_loss,4)}")
print(f"Final validation loss: {round(v_loss,4)}")

Number of parameters: 4134
Final training loss: 0.0007
Final validation loss: 2.9183
